# Traitement des données des actions

In [21]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import plotly.express as px
from functools import reduce 
import plotly.graph_objects as go

In [22]:
#Importer les données
dict_brand = {}
brands=['Apple', 'Adidas', 'Abercrombie', 'Amazon', 'BMW', 'Bosch', 'Dell', 'Microsoft', 'Nike','Zara']
for name in brands:
    dict_brand[name] =  pd.read_csv(name + '.csv', sep=',',decimal=',')
df = pd.read_csv('nb_tweet_h_uyghurs.csv')

In [23]:
#Renommer les colonnes pour différencier les différentes marques
for name in brands : 
    dict_brand[name] = dict_brand[name].rename(columns={'1. open':'open_'+name, '2. high' : 'high_'+name, '3. low' : 'low_'+name, '4. close':'close_'+name}).drop('5. volume', axis=1)

In [24]:
#Faire la conversion de l'index en type date
for name in brands :
    dict_brand[name]['index']=pd.to_datetime(dict_brand[name]['index'], format = "%Y-%m-%d %H:%M:%S")
df['index'] = pd.to_datetime(df['date'], format = '%Y-%m-%d %H:%M:%S')

In [25]:
#Faire la moyenne par  heure pour chaque marque
dict_moyenne={}
for name in brands : 
    dict_moyenne[name]=dict_brand[name][['open_'+name,'high_'+name,'low_'+name,'close_'+name]].astype(float)
    dict_brand[name]['avg_'+name]=dict_moyenne[name].mean(axis=1)

In [26]:
# Ne conserver que les dates auxquelles toutes les valeurs sont non nulles
data_frames = [dict_brand[name] for name in brands]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['index'],how='outer'), data_frames).dropna()
df_merged['index']=pd.to_datetime(df_merged['index'], format = "%Y-%m-%d %H:%M:%S")

In [27]:
fig = go.Figure()

for [name, c] in [['Apple','red'],['Adidas','blue'],['Abercrombie','black'],['Amazon','green'],['BMW','yellow'],['Bosch','pink'],['Dell','purple'],['Microsoft','orange'],['Nike','gold'],['Zara','firebrick']]:
    fig.add_trace(go.Scatter(x=df_merged["index"],y=(df_merged["avg_"+name]-df_merged["avg_"+ name].mean())/pd.DataFrame.std(df_merged["avg_"+ name]),name= name,line=dict(color=c,width=4)))
    df_merged[name+'_normalisé']=(df_merged["avg_"+name]-df_merged["avg_"+ name].mean())/pd.DataFrame.std(df_merged["avg_"+ name])

fig.update_layout(title='Evolution du cours des actions au cours du temps',xaxis_title='Date',yaxis_title='Cours des actions normalisés')

fig.show()

In [28]:
fig = go.Figure()

for [name, c] in [['Apple','red'],['Adidas','blue'],['Abercrombie','black'],['Amazon','green'],['BMW','yellow']]:
    fig.add_trace(go.Scatter(x=df_merged["index"],y=(df_merged["avg_"+name]-df_merged["avg_"+ name].mean())/pd.DataFrame.std(df_merged["avg_"+ name]),name= name,line=dict(color=c,width=4)))
    df_merged[name+'_normalisé']=(df_merged["avg_"+name]-df_merged["avg_"+ name].mean())/pd.DataFrame.std(df_merged["avg_"+ name])
    
fig.update_layout(title='Evolution du cours des actions au cours du temps',xaxis_title='Date',yaxis_title='Cours des actions normalisés')

fig.show()

In [32]:
fig = go.Figure()
for [name, c] in [['Bosch','pink'],['Dell','purple'],['Microsoft','orange'],['Nike','gold'],['Zara','firebrick']]:
    fig.add_trace(go.Scatter(x=df_merged["index"],y=(df_merged["avg_"+name]-df_merged["avg_"+ name].mean())/pd.DataFrame.std(df_merged["avg_"+ name]),name= name,line=dict(color=c,width=4)))
    df_merged[name+'_normalisé']=(df_merged["avg_"+name]-df_merged["avg_"+ name].mean())/pd.DataFrame.std(df_merged["avg_"+ name])

fig.update_layout(title='Evolution du cours des actions au cours du temps',xaxis_title='Date',yaxis_title='Cours des actions normalisés')

fig.show()

# Faire la moyenne par heure des valeurs des 10 actions, pour voir l'évolution moyenne des 10 actions

In [30]:
moyenne10=df_merged[[name+'_normalisé' for name in brands]].astype(float)
df_merged['avg10']=moyenne10.mean(axis=1)

In [31]:
fig = px.line(df_merged, x="index", y='avg10')
fig.show()